# Recursive

In [1]:
from operator import mul
def fact(n):
    if n <= 1:
        return 1
    else:
        return mul(n,fact(n-1))
def fact_iter(n,k=1):
    if n <= 1:
        return k
    else:
        return fact_iter(n-1,k*n)

fact(7),fact_iter(7)

(5040, 5040)

# Data Structure Representation of Continuation

In [2]:
from dataclasses import dataclass
from typing import Callable

@dataclass
class Cont:
    cc:Callable[[int],int]

@dataclass
class End_cc(Cont):
    cc:Callable[[int],int] = None

@dataclass
class Fact_cc(Cont):
    n:int

def fact_cc(cc,n):
    return Fact_cc(cc,n)

def apply_cc(cc:Cont,v:int):
    print(v,'\t',cc)
    if isinstance(cc,End_cc):
        print('end of continuation')
        return v
    elif isinstance(cc,Fact_cc):
        return apply_cc(cc.cc,mul(cc.n,v))
    else:
        print(cc)
        raise NotImplementedError

def fact_k(n,cc):
    if n <= 1:
        return apply_cc(cc,1)
    else:
        return fact_k(n-1,fact_cc(cc,n))
fact_k(7,End_cc())

1 	 Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=7), n=6), n=5), n=4), n=3), n=2)
2 	 Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=7), n=6), n=5), n=4), n=3)
6 	 Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=7), n=6), n=5), n=4)
24 	 Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=7), n=6), n=5)
120 	 Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=7), n=6)
720 	 Fact_cc(cc=End_cc(cc=None), n=7)
5040 	 End_cc(cc=None)
end of continuation


5040

# Registerized

In [3]:
@dataclass
class Registers:
    v:None = None
    n:None = None
    cc:None = None

REG = Registers()

def fact(n):
    REG.n = n
    REG.cc = End_cc()
    fact_k()

def fact_k():
    n = REG.n
    cc = REG.cc
    if n <= 1:
        REG.v = 1
        apply_cc()
    else:
        REG.cc = fact_cc(cc,n)
        REG.n = n - 1
        fact_k()

def apply_cc():
    print(REG)
    cc = REG.cc
    if isinstance(cc,End_cc):
        REG.cc = None
        print('end of continuation')
    elif isinstance(cc,Fact_cc):
        REG.n = cc.n
        REG.v = mul(REG.n,REG.v)
        REG.cc = cc.cc
        apply_cc()
    else:
        print(cc)
        raise NotImplementedError
        
fact(5)

Registers(v=1, n=1, cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=5), n=4), n=3), n=2))
Registers(v=2, n=2, cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=5), n=4), n=3))
Registers(v=6, n=3, cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=5), n=4))
Registers(v=24, n=4, cc=Fact_cc(cc=End_cc(cc=None), n=5))
Registers(v=120, n=5, cc=End_cc(cc=None))
end of continuation


# Registerized and Imperative

In [4]:
from dataclasses import dataclass

@dataclass
class Registers:
    pc:None = None
    v:None = None
    n:None = None
    cc:None = None

REG = Registers()

def fact(n):
    REG.n = n
    REG.cc = End_cc()
    REG.pc = fact_k

def trampoline():
    while REG.pc:
        REG.pc()
    
    return REG.v

def fact_k():
    n = REG.n
    cc = REG.cc
    if n <= 1:
        REG.v = 1
        REG.pc = apply_cc
    else:
        REG.cc = fact_cc(cc,n)
        REG.n = n - 1
        REG.pc = fact_k

def apply_cc():
    print(REG)
    cc = REG.cc
    if isinstance(cc,End_cc):
        REG.cc = None
        REG.pc = False
        print('end of continuation')
    elif isinstance(cc,Fact_cc):
        REG.n = cc.n
        REG.v = mul(REG.n,REG.v)
        REG.cc = cc.cc
        # why still work without this line?
        # REG.pc = apply_cc
    else:
        print(cc)
        raise NotImplementedError
        
fact(5)
trampoline()

Registers(pc=<function apply_cc at 0x0000022CCB175750>, v=1, n=1, cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=5), n=4), n=3), n=2))
Registers(pc=<function apply_cc at 0x0000022CCB175750>, v=2, n=2, cc=Fact_cc(cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=5), n=4), n=3))
Registers(pc=<function apply_cc at 0x0000022CCB175750>, v=6, n=3, cc=Fact_cc(cc=Fact_cc(cc=End_cc(cc=None), n=5), n=4))
Registers(pc=<function apply_cc at 0x0000022CCB175750>, v=24, n=4, cc=Fact_cc(cc=End_cc(cc=None), n=5))
Registers(pc=<function apply_cc at 0x0000022CCB175750>, v=120, n=5, cc=End_cc(cc=None))
end of continuation


120

# Procedural Representation of Continuation

In [5]:
def end_cc(n):
    print('end of continuation')
    return n

def apply_cc(cc,v):
    return cc(v)

def fact_cc(cc,n):
    return lambda v: apply_cc(cc,mul(n,v))

def fact_k(n,cc=end_cc):
    if n <= 1:
        return apply_cc(cc,1)
    else:
        return fact_k(n-1,fact_cc(cc,n))

def fact_iter_k(n,k=1,cc=end_cc):
    if n <= 1:
        return apply_cc(cc,k)
    else:
        return fact_iter_k(n-1,k*n,cc) # tail-call doesn't need new continuation

fact_k(7),fact_iter_k(7)

end of continuation
end of continuation


(5040, 5040)

# Inlined Procedural Continuation

In [6]:
def fact_k(n,cc=end_cc):
    if n <= 1:
        return cc(1)
    else:
        return fact_k(n-1,lambda v: cc(mul(n,v)))
fact_k(7)

end of continuation


5040